# Experiment 3: Time-Aware Baseline Model
Using only hour of day and station ID.

In [2]:
!pip install capymoa

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 30.7 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=bb49c558e9a91aeaeb789c6b53d3ee421eda5795cffda58a4a12b7fdbe2185da
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from capymoa.regressor import FIMTDD
from capymoa.stream import NumpyStream

ImportError: cannot import name 'Instance' from 'capymoa.stream' (/usr/local/lib/python3.11/dist-packages/capymoa/stream/__init__.py)

In [29]:
# Load dataset
df = pd.read_csv('/content/loader_f30avg_new.csv')
df['datetime'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['datetime'].dt.hour
df['station'] = df['stop'].astype(str)

In [30]:
# Prepare features and target
features = df[['hour', 'station']]
target = df['f30avg']

encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(features)

X_array = X_encoded.astype(np.float32)
y_array = target.to_numpy().astype(np.float32)

stream = NumpyStream(X_array, y_array, dataset_name="TimeAware")

In [36]:
# Initialize model and delayed evaluation
model = FIMTDD(stream.get_schema())
true_vals, preds = [], []
buf_X, buf_y = [], []

while stream.has_more_instances():
    inst = stream.next_instance()
    y_i = inst.y_value

    if len(buf_X) >= 6:
        inst_pred = buf_X.pop(0)
        pred = model.predict(inst_pred)
        true_vals.append(buf_y.pop(0))
        preds.append(pred)

    model.train(inst)
    buf_X.append(inst)
    buf_y.append(y_i)

Experiment 3 – MAE: 18.70, RMSE: 44.17


In [37]:
# Results
mae = mean_absolute_error(true_vals, preds)
rmse = sqrt(mean_squared_error(true_vals, preds))
print(f"Experiment 3 — MAE: {mae:.2f}, RMSE: {rmse:.2f}")

Experiment 3 — MAE: 18.70, RMSE: 44.17
